In [1]:
import os
import sys
import time
import psutil
import pandas as pd
import numpy  as np
import datetime
from email_validator import validate_email, EmailNotValidError
from tqdm import tqdm
import concurrent.futures
import multiprocessing as mp

In [4]:
df = pd.read_pickle('nevalid_site_4.pkl')

In [6]:
df = df[df['email'].map(len) > 4].copy()

In [12]:
def check(email):
    try:
        # validate and get info
        valid = validate_email(email)
        # replace with normalized form
        return 'Valid'
    except:
        return 'Not_valid'

In [13]:
# def check(email):
# 	try:
# 	# validate and get info
# 		v = validate_email(email, check_deliverability = False)
# 		# replace with normalized form
# 		email = v["email"]
# 		return "True"
# 	except EmailNotValidError as e:
# 		# email is not valid, exception message is human-readable
# 		return str(e)

In [15]:
def get_validate_email(df):
    df['valid'] = df['email'].map(check)
    return df

In [19]:
df_results = []

In [20]:
# entry point of the program
if __name__ == '__main__':
    logical    = False
    start = time.time()
    num_procs  = psutil.cpu_count(logical=logical)
    splitted_df = np.array_split(df, num_procs)
    # create a process pool with 4 workers
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=num_procs)
    futures = [executor.submit(get_validate_email, df) for df in splitted_df]
    
    # iterate over all submitted tasks and get results as they are available
    for future in tqdm(concurrent.futures.as_completed(futures)):
        try:
            df_results.append(future.result())
        except Exception as ex:
            print(str(ex))
            pass
    end = time.time()
    print("-------------------------------------------")
    print("PPID %s Completed in %s"%(os.getpid(), round(end-start,2)))

8it [13:11, 98.88s/it] 

-------------------------------------------
PPID 15446 Completed in 791.13


In [21]:
df_results = pd.concat(df_results)

In [22]:
df_results

,site_id,user_id,email,email_confirm,valid
105018,4,5300111,test-user-1157045832@test.qa,0,Not_valid
105019,4,5300113,test-user-652405130@test.qa,0,Not_valid
105020,4,5300115,test-user-1520529720@test.qa,0,Not_valid
105021,4,5300117,test-user-1867456334@test.qa,0,Not_valid
105022,4,5300119,test-user-1245260547@test.qa,0,Not_valid
...,...,...,...,...,...
401149,4,891872,ali.zar@yandex.com,1,Valid
401197,4,5565609,u.tutkushev@ttc.kz,0,Valid
401212,4,5016604,dingizbaevkurbanmurad@yandex.kz,0,Valid
401231,4,5181046,symbattashanova@mail.com,0,Valid


In [23]:
df_results.query("valid == 'Not_valid'")['valid'].value_counts()

Not_valid    15478
Name: valid, dtype: int64

In [24]:
df_results.query("valid == 'Not_valid'")

,site_id,user_id,email,email_confirm,valid
105018,4,5300111,test-user-1157045832@test.qa,0,Not_valid
105019,4,5300113,test-user-652405130@test.qa,0,Not_valid
105020,4,5300115,test-user-1520529720@test.qa,0,Not_valid
105021,4,5300117,test-user-1867456334@test.qa,0,Not_valid
105022,4,5300119,test-user-1245260547@test.qa,0,Not_valid
...,...,...,...,...,...
399783,4,4548853,zenebe@gmail.com.co.com,0,Not_valid
400480,4,677720,bekzodjonsuvonov@gimail.com,0,Not_valid
400569,4,2428039,dtoskina@mail.ua,0,Not_valid
400678,4,5107651,iuIya.malaai@gmaiI.com,0,Not_valid
